<div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;">
    <h2 style="color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
        Анализ данных: Основные результаты
    </h2>
    <p style="color: #34495e; line-height: 1.6;">
        Ниже представлены результаты проведенного анализа данных. Вы можете ознакомиться с подробными отчетами, перейдя по ссылкам:
    </p>
    <div style="margin: 20px 0; padding: 15px; background-color: white; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
        <h3 style="color: #2980b9; margin-bottom: 15px;">Доступные отчеты:</h3>
        <div style="margin-bottom: 15px;">
            <a href="https://drive.google.com/file/d/17WrH5_gFN5mbX6-0RcGBkfxGz99k4R8O/view?usp=sharing" 
               style="color: #3498db; text-decoration: none; display: block; padding: 10px; border: 1px solid #bdc3c7; border-radius: 5px; margin-bottom: 10px;">
                Отчет №1: Анализ данных - производство бумаги
            </a>
            <a href="https://drive.google.com/file/d/1lRXPlpBAzzCawFOsEQBXjrZc1i-ZPI0n/view?usp=sharing" 
               style="color: #3498db; text-decoration: none; display: block; padding: 10px; border: 1px solid #bdc3c7; border-radius: 5px;">
                Отчет №2: Анализ данных веб-скрайпинг (получение данных с веб-сайтов)
            </a>
        </div>
    </div>
    <p style="color: #7f8c8d; font-size: 0.9em;">
        Ниже вы найдете полный код Python с комментариями процесса анализа данных.
    </p>
</div>
<!-- Вакансия – https://hh.ru/vacancy/116099143?hhtmFrom=chat       link - https://disk.yandex.ru/i/vweb3RtSazZtPg -->

<div style="height: 80px;"></div>  <!-- 5см = 190px -->


<h1>Анализ рынка бумаги</h1>

<h2>1. Структура и динамика производства бумаги</h2>

<h3>1.1 Региональная структура производства бумаги и её изменение</h3>
<h3>1.2 Помесячная динамика производства бумаги в 2010-2011гг</h3>
<h3>1.3 Сравнительный анализ динамики производства бумаги 2011г. с 2010г</h3>
<h3>1.4 Графическая визуализация результатов анализа</h3>

<p><strong>Источники данных:</strong></p>
<ul>
    <li>Таблица 1.1 Производство бумаги в РФ, 2007-2010 гг.</li>
    <li>Таблица 1.2 Производство бумаги в РФ по федеральным округам, 2010-2011 гг., тонн</li>
</ul>


In [ ]:
from fpdf import FPDF
from datetime import datetime
import os
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib.units import mm, cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from datetime import datetime
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph
import requests
from bs4 import BeautifulSoup
import pandas as pd
from typing import Dict, List, Tuple
import re

In [ ]:
class ReportPDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_margins(left=30, top=15, right=30)
        self.set_auto_page_break(auto=True, margin=15)
        self.add_font('DejaVu', '', 'font/DejaVuSansMono.ttf', uni=True)
        self.add_font('DejaVu', 'B', 'font/DejaVuSansMono-Bold.ttf', uni=True)
        self.page_no = 0

    def header(self):
        self.page_no += 1
        if self.page_no > 1:
            self.set_font('DejaVu', 'B', 15)
            self.cell(0, 10, 'Анализ производства бумаги в России', align='C')
            self.ln(15)

    def chapter_title(self, title):
        self.set_font('DejaVu', 'B', 12)
        self.cell(0, 10, title, ln=True)
        self.ln(3)

    def chapter_body(self, body):
        self.set_font('DejaVu', '', 11)
        self.multi_cell(0, 4, body)
        self.ln()

    def add_colored_box(self, text, color, share=None):
        # Сохраняем текущие цвета
        current_fill = self.fill_color
        current_text = self.text_color
        
        # Рисуем цветной квадратик
        self.set_fill_color(*color)
        self.cell(5, 5, '', 1, 0, 'L', True)
        self.set_fill_color(int(current_fill[0:2], 16), int(current_fill[2:4], 16), int(current_fill[4:6], 16))
        
        # Печатаем текст
        self.cell(5, 5, '')  # Отступ после квадратика
        if share is not None:
            self.cell(0, 5, f"{text}: {share}%", ln=True)
        else:
            self.cell(0, 5, text, ln=True)

    def add_image(self, img_path, caption):
        page_width = 210
        margins = self.l_margin + self.r_margin
        img_width = page_width - margins - 10
        
        self.image(img_path, x=self.l_margin + 5, w=img_width)
        self.set_font('DejaVu', '', 10)
        self.cell(0, 10, caption, align='C', ln=True)
        self.ln(5)

In [ ]:
def generate_report(analysis_results):
    regions_2010 = analysis_results['regions_2010']
    regions_2010_sorted = analysis_results['regions_2010_sorted']
    top_3_share = analysis_results['top_3_share']
    yoy_changes = analysis_results['yoy_changes']
    regional_stats = analysis_results['regional_stats']
    
    pdf = ReportPDF()
    
    # Титульная страница
    pdf.add_page()
    pdf.set_font('DejaVu', '', 12)  # Маленький шрифт для даты
    current_date = datetime.now().strftime("%d.%m.%Y")
    combined_text = f"Отчёт выполнен {current_date}, Ганнова Анфиса"  # или любой другой текст
    pdf.cell(0, 10, combined_text, align='R', ln=True)
    
    # Основной заголовок
    pdf.set_font('DejaVu', 'B', 20)
    pdf.ln(60) # Отступ сверху
    pdf.cell(0, 10, 'Анализ производства бумаги', align='C', ln=True)
    
    # Дата в правом нижнем углу
    pdf.set_font('DejaVu', '', 8) # Маленький шрифт для даты
    
    #-------- Региональная структура
    pdf.add_page()
    pdf.chapter_title('1. Региональная структура производства')
    pdf.add_image('temp_figures/structure.png', 'Рис. 1: Структура производства бумаги по регионам (2010)')
    
    pdf.chapter_body('\nКонцентрация производства:')
    pdf.chapter_body(f'• Топ-3 региона контролируют {top_3_share:.1f}% производства')
    pdf.chapter_body('\nДоли регионов в общем объеме производства (2010):')
    for _, region in regions_2010_sorted.iterrows():
        pdf.chapter_body(f"• {region['Округ']}: {region['Доля_2010']}%")
    
    # Динамика производства
    pdf.add_page()
    pdf.chapter_title('2. Динамика производства 2007-2010')
    pdf.add_image('temp_figures/dynamics.png', 'Рис. 2: Динамика производства бумаги (2007-2010)')
    
    pdf.chapter_body('\nИзменения в 2010 году по сравнению с 2009:')
    for _, region in regions_2010.iterrows():
        change = region['Изменение_2009_2010']
        if isinstance(change, str):
            pdf.chapter_body(f"• {region['Округ']}: {change}")
        else:
            pdf.chapter_body(f"• {region['Округ']}: {change:+.1f}%")
    
    pdf.chapter_body('\nСреднегодовой темп роста 2007-2010:')
    for _, region in regions_2010.iterrows():
        if pd.notna(region['CAGR_2007_2010']):
            pdf.chapter_body(f"• {region['Округ']}: {region['CAGR_2007_2010']:+.1f}%")
    
    # Помесячная динамика
    pdf.add_page()
    pdf.chapter_title('3. Помесячная динамика 2010-2011')
    pdf.add_image('temp_figures/monthly.png', 'Рис. 3: Помесячная динамика производства (2010-2011)')
    
    # Сравнение 2011 с 2010
    pdf.chapter_body('\nИзменение объемов производства 2011 к 2010:')
    for region, change in yoy_changes.items():
        pdf.chapter_body(f"• {region}: {change:+.1f}%")
    
    # Статистика по регионам
    pdf.chapter_body('\nОсновные статистические показатели:')
    for region, stats in regional_stats.items():
        pdf.chapter_body(f"\n{region}:")
        pdf.chapter_body(f"• Среднемесячный объем: {stats['mean']:,.0f}")
        pdf.chapter_body(f"• Коэффициент вариации: {stats['cv']}%")
    
    # Выводы
    pdf.add_page()
    pdf.chapter_title('4. Основные выводы')
    pdf.chapter_body("""
1. Региональная структура производства:
• Высокая концентрация производства в ключевых регионах
• Появление новых производственных мощностей в Дальневосточном ФО
• Значительное усиление позиций Центрального и Южного ФО

2. Динамика производства 2007-2010:
• Общий тренд на рост производства в 2010 году
• Различная динамика роста по регионам
• Стабильное производство в крупнейших регионах
• Высокая волатильность в малых регионах

3. Сравнительный анализ 2010-2011:
• Изменение объемов производства по регионам
• Сезонные колебания производства
• Различная динамика роста в разных регионах

4. Ключевые тенденции:
• Укрепление позиций традиционных центров производства
• Развитие новых производственных центров
• Общий тренд на рост производства
    """)
    
    
    # Сохранение отчета
    pdf.output('report_0.pdf')
    
    # Удаление временных файлов
    for file in os.listdir('temp_figures'):
        os.remove(f'temp_figures/{file}')
    os.rmdir('temp_figures')

In [ ]:
if __name__ == "__main__":
    analysis_results = main(df_10_11)
    generate_report(analysis_results)

<div style="height: 120px;"></div>  <!-- 5см = 190px -->

<h1>Анализ компаний алюминиевой промышленности</h1>

<h2>Извлечение сведений о компаниях на основании данных сайтов</h2>

<h3>1. Century Extrusions Ltd.</h3>
<p><strong>Источник данных:</strong> <a href="https://www.centuryextrusions.com">www.centuryextrusions.com</a></p>

<h3>2. Maan Aluminium Ltd.</h3>
<p><strong>Источник данных:</strong> <a href="https://www.maanaluminium.com">www.maanaluminium.com</a></p>


#### собираем данные с сайтов по ключевым словам по каждой тематике

In [ ]:
# класс по сбору данных
class KeywordBasedScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        self.keywords = {
    'Общие сведения': ['company', 'about', 'established', 'founded', 'history', 'overview', 'profile', 'background', 'introduction', 
                       'organization', 'entity', 'corporation', 'enterprise', 'institution', 'establishment', 'inception', 'origin', 
                       'formation', 'foundation', 'genesis', 'creation', 'emergence', 'beginning', 'startup', 'commencement', 
                       'inauguration', 'initiation', 'roots', 'legacy', 'heritage', 'tradition', 'story', 'chronicle', 'narrative', 
                       'timeline', 'milestones', 'achievements', 'journey', 'evolution'],
    'Месторасположение': ['located', 'location', 'address', 'facility', 'plant', 'factory', 'headquarter', 'based in', 'situated', 
                          'premises', 'site', 'venue', 'territory', 'region', 'area', 'district', 'zone', 'position', 'placement', 
                          'coordinates', 'geographical', 'spot', 'setting', 'whereabouts', 'domiciled', 'stationed', 'established', 
                          'campus', 'building', 'complex', 'industrial park', 'office', 'branch', 'subsidiary', 'division', 'unit', 
                          'operations center', 'distribution center', 'logistics hub'],
    'Оборудование': ['equipment', 'machinery', 'press', 'furnace', 'production line', 'manufacturing unit', 'facility', 'machine', 
                     'technology', 'apparatus', 'hardware', 'tools', 'devices', 'instruments', 'systems', 'setups', 'installations', 
                     'appliances', 'gear', 'mechanisms', 'contraptions', 'utilities', 'implements', 'operational assets', 
                     'technical infrastructure', 'automation', 'robotics', 'conveyor', 'extruder', 'mixer', 'crusher', 'cutter', 'lathe', 
                     'mold', 'die', 'pump', 'motor', 'compressor', 'generator', 'controller', 'sensor', 'industrial tools'],
    'Ассортимент': ['products', 'produce', 'manufacture', 'range', 'portfolio', 'offering', 'solutions', 'applications', 'segments', 
                    'catalog', 'inventory', 'lineup', 'selection', 'variety', 'assortment', 'collection', 'series', 'array', 'spectrum', 
                    'diversity', 'goods', 'items', 'commodities', 'merchandise', 'stock', 'supply', 'output', 'production', 'creations', 
                    'deliverables', 'options', 'alternatives', 'choices', 'variations', 'models', 'types', 'categories', 'classes', 'groups',
                    'families', 'generations', 'editions', 'versions', 'variants'],
    'Сплавы': ['alloy', 'series', '6061', '6063', '6082', '7075', 'grade', 'composition', 'material', 'metal', 'mixture', 'blend', 'compound',
               
               'formulation', 'combination', 'amalgamation', 'fusion', 'integration', 'aluminum', 'steel', 'titanium', 'bronze', 'brass', 
               'copper', 'nickel', 'zinc', 'magnesium', 'chromium', 'molybdenum', 'tungsten', 'vanadium', 'element', 'component', 
               'constituent', 'ingredient', 'proportion', 'ratio', 'percentage', 'purity', 'standard', 'specification', 'property', 
               'characteristic', 'attribute', 'quality', 'performance', 'strength', 'durability', 'corrosion-resistance', 'heat-resistance', 
               'ductility', 'malleability'],
    'Мощности': ['capacity', 'production', 'output', 'volume', 'tonnage', 'capability', 'annual', 'monthly', 'metric', 'throughput', 'yield', 
                 'performance', 'productivity', 'efficiency', 'rate', 'speed', 'turnover', 'processing', 'handling', 'manufacturing potential',
                 'operational scale', 'industrial strength', 'maximum load', 'utilization', 'bandwidth', 'threshold', 'ceiling', 'limit', 
                 'range', 'scope', 'extent', 'magnitude', 'dimension', 'scale', 'size', 'amount', 'quantity', 'quota', 'allocation', 
                 'measurement', 'calculation', 'estimation', 'projection', 'forecast', 'target', 'goal', 'objective', 'benchmark', 'standard'],
    'Планы развития': ['expansion', 'growth', 'future', 'plan', 'development', 'investment', 'strategy', 'vision', 'upcoming', 'roadmap', 
                       'projection', 'forecast', 'outlook', 'prospect', 'potential', 'opportunity', 'initiative', 'proposal', 'program', 
                       'scheme', 'project', 'venture', 'undertaking', 'endeavor', 'mission', 'objective', 'goal', 'target', 'intention', 
                       'aim', 'purpose', 'direction', 'course', 'path', 'route', 'trajectory', 'blueprint', 'framework', 'structure', 
                       'design', 'layout', 'arrangement', 'configuration', 'construction', 'implementation', 'execution', 'realization', 
                       'advancement', 'progress', 'improvement', 'enhancement', 'upgrade', 'modernization', 'innovation', 'transformation', 
                       'evolution', 'adaptation', 'adjustment', 'modification', 'alteration', 'change', 'shift', 'transition']
}
    def clean_text(self, text: str) -> str:
        """Очищает текст от лишних пробелов и переносов строк."""
        # Заменяем множественные пробелы и переносы строк на одиночные
        text = re.sub(r'\s+', ' ', text)
        # Удаляем пробелы в начале и конце
        text = text.strip()
        # Удаляем повторяющиеся точки
        text = re.sub(r'\.{2,}', '.', text)
        return text

    def get_page_content(self, url: str) -> BeautifulSoup:
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Ошибка при загрузке {url}: {str(e)}")
            return None

    def find_context(self, text: str, keyword: str, window: int = 200) -> List[str]:
        """Находит контекст вокруг ключевого слова."""
        text = text.lower()
        keyword = keyword.lower()
        
        matches = []
        start = 0
        while True:
            index = text.find(keyword, start)
            if index == -1:
                break
                
            context_start = max(0, index - window)
            context_end = min(len(text), index + len(keyword) + window)
            
            context = text[context_start:context_end]
            # Добавляем многоточие, если контекст обрезан
            if context_start > 0:
                context = f"...{context}"
            if context_end < len(text):
                context = f"{context}..."
                
            # Очищаем найденный контекст
            context = self.clean_text(context)
            
            if context not in matches:  # Избегаем дубликатов
                matches.append(context)
            start = index + 1
            
        return matches

    def scrape_website(self, url: str, company_name: str) -> Dict[str, List[str]]:
        results = {category: [] for category in self.keywords.keys()}
        
        soup = self.get_page_content(url)
        if not soup:
            return results
            
        # Получаем весь текст со страницы
        text = ' '.join([p.get_text() for p in soup.find_all(['p', 'div', 'section'])])
        text = self.clean_text(text)
        
        # Ищем контекст для каждой категории
        for category, keywords in self.keywords.items():
            for keyword in keywords:
                contexts = self.find_context(text, keyword)
                if contexts:
                    results[category].extend(contexts)
                    
        return results

    def create_dataframe(self, data: Dict[str, Dict[str, List[str]]]) -> pd.DataFrame:
        """Создает DataFrame из собранных данных."""
        all_data = []
        
        for company, categories in data.items():
            for category, contexts in categories.items():
                # Объединяем все контексты для категории в один текст
                combined_context = ' | '.join(contexts) if contexts else ''
                all_data.append({
                    'Компания': company,
                    'Категория': category,
                    'Найденный контекст': combined_context
                })
        
        return pd.DataFrame(all_data)

In [ ]:
def main():
    scraper = KeywordBasedScraper()
    
    companies = {
        'Century Extrusions': 'https://www.centuryextrusions.com',
        'Maan Aluminium': 'https://www.maanaluminium.com'
    }
    
    all_results = {}
    
    for company_name, url in companies.items():
        print(f"Сбор данных для {company_name}...")
        results = scraper.scrape_website(url, company_name)
        all_results[company_name] = results
    
    # Создаем DataFrame
    df = scraper.create_dataframe(all_results)
    
    # Для удобства просмотра длинных текстов
    pd.set_option('display.max_colwidth', None)
    
    return df

In [41]:
if __name__ == "__main__":
    df = main()

Сбор данных для Century Extrusions...
Сбор данных для Maan Aluminium...


### Данные собраны
#### Сохраняем данные

In [291]:
# Cписки компаний и вопросов
companies = ['Century Extrusions', 'Maan Aluminium']
questions = [ 'Общие сведения', 'Месторасположение', 'Оборудование', 'Ассортимент', 'Сплавы', 'Мощности', 'Планы развития']

In [ ]:
# Функция запроса
def run_query(companies, questions, company_idx, question_idx):
    company = companies[company_idx]
    question = questions[question_idx]
    
    result = df[
        (df['Компания'] == company) & 
        (df['Категория'] == question)
    ]['Найденный контекст'].tolist()
    
    return result

In [ ]:
# Создаем PDF документ
pdf_filename = "scripe.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=A4)
story = []  # Список элементов для добавления в PDF

In [ ]:
# Регистрируем шрифт DejaVu
pdfmetrics.registerFont(TTFont('DejaVu', 'font/dejavu.ttf'))
pdfmetrics.registerFont(TTFont('DejaVu_bold', 'font/DejaVuSansMono-Bold.ttf'))
styles = getSampleStyleSheet()
styles['Normal'].fontName = 'DejaVu'
styles['Heading1'].fontName = 'DejaVu_bold'
styles['Heading1'].fontSize = 14
styles['Heading2'].fontName = 'DejaVu'
styles['Heading2'].fontSize = 12

In [ ]:
# Добавляем заголовок
story.append(Paragraph('<b>Cобранные данные</b>', styles['Heading1']))

In [ ]:
# Проходим по спискам и добавляем результаты в PDF
for company_idx, company in enumerate(companies):
    # Добавляем название компании
    story.append(Paragraph(f'<b>Компания: {company}</b>', styles['Heading2']))
    
    for question_idx, question in enumerate(questions):
        # Получаем результат
        result = run_query(companies, questions, company_idx, question_idx)
        
        # Добавляем вопрос
        story.append(Paragraph(f'<b>Вопрос: {question}</b>', styles['Heading2']))
        
        # Добавляем результаты
        if not result:
            story.append(Paragraph('Нет данных', styles['Normal']))
        else:
            for item in result:
                story.append(Paragraph(f'- {item}', styles['Normal']))
        
        # Добавляем пространство между вопросами
        story.append(Paragraph('<br/>', styles['Normal']))

In [338]:
# Сохраняем PDF
doc.build(story)
print(f"PDF сохранен как {pdf_filename}")

PDF сохранен как scraping.pdf


<div style="margin: 20px 0; padding: 15px; background-color: white; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
        <div style="margin-bottom: 15px;">
            <a href="https://drive.google.com/file/d/1ldYZoMnu1sEQtzx0llyV6E8GsEsjpHdO/view?usp=sharing" 
               style="color: #3498db; text-decoration: none; display: block; padding: 10px; border: 1px solid #bdc3c7; border-radius: 5px; margin-bottom: 10px;">
                Собранные данные с сайтов
            </a>
        </div>
    </div>

# Создаем PDF с результатами

In [ ]:
def create_company_report(filename="company_analysis_report.pdf", author="Анна Иванова", left_indent=5*mm):
    """
    Создает PDF-отчет по анализу компаний с использованием шрифта DejaVu.
    """
    # Регистрируем шрифт DejaVu
    try:
        pdfmetrics.registerFont(TTFont('DejaVu', 'font/dejavu.ttf'))
    except:
        print("Предупреждение: Файл шрифта не найден. Используем стандартные шрифты.")
        pdfmetrics.registerFont(TTFont('DejaVu', pdfmetrics._fonts['Helvetica']._file))
    
    # Создаем PDF документ
    doc = SimpleDocTemplate(filename, 
                          pagesize=A4, 
                          rightMargin=25*mm,
                          leftMargin=25*mm,
                          topMargin=20*mm, 
                          bottomMargin=20*mm)
    
    # Получаем стандартные стили
    styles = getSampleStyleSheet()
    
    # Стиль для даты и автора на титульном листе
    styles.add(ParagraphStyle(
        name='DateAuthor',
        fontName='DejaVu',
        fontSize=10,
        alignment=2,  # По правому краю
        spaceAfter=30
    ))
    
    # Стиль для заголовка на титульном листе
    styles.add(ParagraphStyle(
        name='TitlePage',
        fontName='DejaVu',
        fontSize=20,
        alignment=1,  # По центру
        spaceAfter=0,
        leading=30
    ))
    
    # Создаем единый стиль для заголовка компании
    styles.add(ParagraphStyle(
        name='CompanyHeader',
        fontName='DejaVu',
        fontSize=14,
        alignment=1,
        spaceAfter=15,
        leading=20
    ))
    
    # Базовый стиль для всего текста с отступом
    styles.add(ParagraphStyle(
        name='BaseText',
        fontName='DejaVu',
        fontSize=10,
        alignment=0,
        leftIndent=left_indent,
        spaceAfter=5
    ))
    
    # Стиль для заголовков секций
    styles.add(ParagraphStyle(
        name='SectionHeader',
        fontName='DejaVu',
        fontSize=12,
        textColor=colors.HexColor('#1a237e'),
        spaceAfter=5
    ))

    # Данные компаний
    companies = [
        {
            "name": "CENTURY EXTRUSIONS LIMITED",
            "link": "https://www.centuryextrusions.com",
            "sections": {
                "Общие сведения": [
                    "История и основание: Компания основана в 1988 году, основатель - покойный Шри М.П.Джунджунвала (M.P.Jhunjhunwala), предприниматель первого поколения",
                    "Специализация: Один из крупнейших производителей алюминиевого экструзионного профиля в Индии, специализируется на производстве экструдированной и протянутой продукции",
                    "Преимущества: Сильная клиентская база, репутация надежного бренда, низкая историческая стоимость активов",
                    "Комплексное понимание рынка алюминия и алюминиевых экструзий",
                    "Отзывы клиентов: Профессиональная организация с хорошим отношением к клиентам, высокая оценка от Siddhi Engineers и Eaton Fluid Power",
                    "Финансы: Публично торгуется на бирже, регулярная публикация финансовых результатов",
                    "Контакты: +91 33 2229 1012, +91 33 2229 1291",
                    "Email: works@centuryextrusions.com, enquiry@centuryextrusions.com"
                ],
                "Местоположение": [
                    "Головной офис: 113, Park Street, 'N' Block, 2-й этаж, Калькутта (Kolkata) – 700 016, Индия",
                    "Производство: WBIIDC Industrial Growth Centre, Plot No. 7A, Sector 'B', Nimpura, P.O.- Rakhajungle, Kharagpur – 721 301, (W.B), Индия",
                    "Стратегическое расположение близко к ведущим производителям первичного алюминия в Индии"
                ],
                "Оборудование": [
                    "Прессовое оборудование: Экструзионные прессы и прессовые линии",
                    "Погрузочно-разгрузочное оборудование для обработки материалов",
                    "Системы контроля качества",
                    "Отличное техническое оснащение с высококачественным производственным оборудованием",
                    "Современное оборудование с правильным подходом к производству",
                    "Хорошие технические навыки персонала с системой контроля качества продукции"
                ],
                "Ассортимент": [
                    "Экструдированные и тянутые изделия",
                    "Изделия для энергетики и распределительных сетей (T&D Hardware)",
                    "Промышленные и архитектурные применения",
                    "Транспортные и структурные компоненты",
                    "Широкий выбор сплавов с различными свойствами",
                    "Применение в строительстве, автомобилестроении, солнечной энергетике, оборонной промышленности, железнодорожной отрасли, электротехнике, машиностроении и медицине"
                ],
                "Сплавы": [
                    "Серия 1000: 1050 (19500), 1060 (19600), 1070, 1100 (19000), 1350 (19501)",
                    "Серия 2000: 2011, 2014 (24345), 2017, 2024",
                    "Серия 3000: 3003 (31000), 3103",
                    "Серия 4000: 4043, 4047 для сварочного применения",
                    "Серия 5000: 5052 (52000), 5083 (54300), 5086 (53000), 5356",
                    "Серия 6000: 6005/6005A, 6060, 6082"
                ],
                "Мощности": [
                    "35-летний опыт работы в отрасли",
                    "Стратегическое расположение рядом с производителями первичного алюминия",
                    "Современная производственная инфраструктура с несколькими экструзионными прессами",
                    "Полный производственный цикл с системами контроля качества",
                    "Низкая историческая стоимость активов"
                ],
                "Планы развития": [
                    "Постоянное совершенствование производственных мощностей",
                    "Расширение ассортимента алюминиевых сплавов",
                    "Развитие технической поддержки клиентов",
                    "Внедрение систем экологического менеджмента",
                    "Акцент на безопасность производства и охрану труда",
                    "Диверсификация отраслей применения продукции"
                ]
            }
        },
        {
            "name": "MAAN ALUMINIUM LTD",
            "link": "https://www.maanaluminium.com",
            "sections": {
                "Общие сведения": [
                    "Организационная структура: Публичная компания с системой корпоративного управления, существует как минимум с 2017 года",
                    "Основные направления: Производство алюминиевых изделий, торговля слитками, заготовками и цветным металлоломом",
                    "Технологические процессы: Специализированная термическая обработка, использование печей для нагрева экструзий",
                    "Структура компании: Производство, контроль качества, торговое подразделение, корпоративное управление, отдел по работе с инвесторами",
                    "Корпоративное управление: Регулярная публикация отчетов, программа корпоративной социальной ответственности (CSR)",
                    "Особенности: Собственный каталог продукции, система управления качеством, активная программа найма сотрудников"
                ],
                "Местоположение": [
                    "Корпоративный офис: 4/5, 1-й этаж, Asaf Ali Road, Нью-Дели (New Delhi) - 110002, Индия",
                    "Производственное предприятие: Plot No-67 & 75, Sector 1, Pithampur, Dhar, Мадхья-Прадеш (Madhya Pradesh) - 454 775, Индия",
                    "Стратегическое расположение: Корпоративный офис в столице Индии, производство в промышленном районе штата Мадхья-Прадеш"
                ],
                "Оборудование": [
                    "Экструзионное оборудование: Пресс высокого давления, нагревательная печь для биллетов",
                    "Системы обработки: Охлаждающая система (воздух/вода), растяжной стол, конвейерная пила",
                    "Термообработка: Печи для термической обработки (старения)",
                    "Процесс производства: Подготовка материала, экструзия, пост-обработка",
                    "Возможности: Работа с заготовками до 7 метров, полный цикл производства",
                    "Контроль качества: Современные системы контроля на всех этапах производства"
                ],
                "Ассортимент": [
                    "Основная продукция: Алюминиевые экструзионные профили, слитки и биллеты, цветной металлолом",
                    "Характеристики: Длина заготовок до 7 метров, широкий ассортимент сплавов и диаметров",
                    "Преимущества продукции: Низкий вес, высокая прочность, превосходная пластичность",
                    "Дополнительные свойства: Легкость в обработке, коррозионная стойкость, хорошая тепло- и электропроводность",
                    "Экологичность: 100% пригодность к переработке, 75% произведенного алюминия все еще в использовании",
                    "Области применения: Строительство, автомобилестроение, солнечная энергетика, оборонная промышленность, железнодорожная отрасль"
                ],
                "Сплавы": [
                    "Характеристики сплавов: Широкий диапазон составов и диаметров, высокая прочность",
                    "Технические свойства: Хорошая коррозионная стойкость, высокая теплопроводность и электропроводность",
                    "Физические свойства: Малый вес, превосходная пластичность",
                    "Производственный процесс: Предварительный нагрев, экструзия, термическая обработка",
                    "Контроль качества: Тщательный контроль производства и термообработки",
                    "Экологичность: Бесконечная пригодность к переработке, универсальность применения"
                ],
                "Мощности": [
                    "Производственные процессы: Экструзия, предварительный нагрев, охлаждение, растяжка и резка профилей",
                    "Технические возможности: Работа с заготовками до 7 метров, различные диаметры профилей",
                    "Оборудование: Экструзионные прессы, нагревательные печи, системы охлаждения, растяжное оборудование",
                    "Направления: Промышленные профили, торговля слитками, переработка металлолома",
                    "Особенности: Современные технологии экструзии, контроль качества, гибкость производства"
                ],
                "Планы развития": [
                    "Диверсификация: Расширение присутствия в различных отраслях промышленности",
                    "Технологическое развитие: Совершенствование процессов термообработки, улучшение свойств экструзии",
                    "Экологическая устойчивость: Акцент на переработке алюминия, повышение уровня повторного использования",
                    "Качество: Развитие системы корпоративного управления, поддержание высоких стандартов",
                    "Расширение: Потенциал для увеличения производственных мощностей"
                ]
            }
        }
    ]
    
    # Содержимое документа
    content = []
    
    # Добавляем титульный лист
    current_date = datetime.now().strftime("%d.%m.%Y")
    content.append(Paragraph(f"{current_date}<br/>{author}", styles['DateAuthor']))
    content.append(Spacer(1, 50*mm))
    content.append(Paragraph("Анализ компаний<br/>алюминиевой промышленности", styles['TitlePage']))
    content.append(PageBreak())
    
    # Перебираем компании
    for i, company in enumerate(companies):
        # Создаем единую строку заголовка с разными размерами шрифта и голубой подчеркнутой ссылкой
        header = f'''<font size="15">Досье компании</font> - 
                    <font size="14">{company["name"]}</font> - 
                    <font size="12" color="#4285f4"><u><link href="{company["link"]}">{company["link"]}</link></u></font>'''
        content.append(Paragraph(header, styles['CompanyHeader']))
        content.append(Spacer(1, 10*mm))
        
        # Перебираем секции
        for section_name, section_items in company["sections"].items():
            # Добавляем заголовок секции
            content.append(Paragraph(section_name, styles['SectionHeader']))
            
            # Создаем таблицу для элементов
            data = []
            for item in section_items:
                data.append([Paragraph(item, styles['BaseText'])])
            
            table = Table(data, colWidths=[450])
            table.setStyle(TableStyle([
                ('FONTNAME', (0, 0), (-1, -1), 'DejaVu'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
                ('TOPPADDING', (0, 0), (-1, -1), 6),
                ('LINEBELOW', (0, 0), (-1, -2), 0.25, colors.lightgrey),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('LEFTPADDING', (0, 0), (-1, -1), 5),
                ('RIGHTPADDING', (0, 0), (-1, -1), 5),
            ]))
            
            content.append(table)
            content.append(Spacer(1, 8*mm))
        
        # Добавляем разрыв страницы между компаниями
        if i < len(companies) - 1:
            content.append(PageBreak())
    
    # Создаем PDF документ
    doc.build(content)
    print(f"Отчет успешно создан: {filename}")

In [325]:
# Вызываем функцию для создания отчета
if __name__ == "__main__":
    create_company_report("report_1.pdf", author="Ганнова Анфиса", left_indent=5*mm)

Отчет успешно создан: report_1.pdf
